In [2]:
from finance_complaint.config.spark_manager import spark_session

:: loading settings :: url = jar:file:/usr/local/spark-3.2.1-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /config/.ivy2/cache
The jars for the packages stored in: /config/.ivy2/jars
com.amazonaws#aws-java-sdk added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7e1cd99c-a9f7-4d06-a459-449e064b52a6;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk;1.7.4 in central
	found commons-logging#commons-logging;1.1.1 in central
	found org.apache.httpcomponents#httpclient;4.2 in central
	found org.apache.httpcomponents#httpcore;4.2 in central
	found commons-codec#commons-codec;1.3 in central
	found com.fasterxml.jackson.core#jackson-core;2.1.1 in central
	found com.fasterxml.jackson.core#jackson-databind;2.1.1 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.1.1 in central
	found joda-time#joda-time;2.12.2 in central
	[2.12.2] joda-time#joda-time;[2.2,)
	found org.apache.hadoop#hadoop-aws;2.7.3 in central
	found org.apache.hadoop#hadoop-common;2.7.3 in central
	found

In [14]:
df = spark_session.read.parquet("/config/workspace/finance_artifact/data_ingestion/feature_store/finance_complaint")

In [15]:
df.count()

379989

In [16]:
df.printSchema()

root
 |-- company: string (nullable = true)
 |-- company_public_response: string (nullable = true)
 |-- company_response: string (nullable = true)
 |-- complaint_id: string (nullable = true)
 |-- complaint_what_happened: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- consumer_disputed: string (nullable = true)
 |-- date_received: string (nullable = true)
 |-- date_sent_to_company: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- product: string (nullable = true)
 |-- state: string (nullable = true)
 |-- sub_issue: string (nullable = true)
 |-- sub_product: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- timely: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [18]:
for column in df.columns:
    print(f"{column}:{df.select(column).distinct().count()}")

company:2458
company_public_response:11
company_response:6


complaint_id:369751


complaint_what_happened:97281
consumer_consent_provided:6
consumer_disputed:1
date_received:174
date_sent_to_company:174
issue:77
product:9
state:60
sub_issue:163
sub_product:49
submitted_via:5
tags:4
timely:2
zip_code:16458


In [24]:
sql = spark_session.sql

In [25]:
complaint_table="complaint"
df.createOrReplaceTempView(complaint_table)

In [26]:
def perform_null_analysis(df,table_name):
    null_value_analysis=[]
    n_row=df.count()
    for column in df.columns:
        print(column)
        response = sql(f"select {n_row} as  total_row,count(*) as null_row_{column},(count(*)*100)/{n_row} as missing_percentage,'{column}' as  column_name from {table_name} where {column} is null").collect()
        null_value_analysis.append(response)
    return null_value_analysis

In [27]:
null_report = perform_null_analysis(df,complaint_table)

company
company_public_response
company_response
complaint_id
complaint_what_happened
consumer_consent_provided
consumer_disputed
date_received
date_sent_to_company
issue
product
state
sub_issue
sub_product
submitted_via
tags
timely
zip_code


In [28]:
def unwanted_column_by_missing_percentage(null_value_analysis,per_thres=20):
    columns= []
    for row in null_value_analysis:
        row_info=row[0]
        if row_info.missing_percentage>per_thres:
            print(row_info)
            columns.append(row_info.column_name)

    return columns

In [29]:
columns = unwanted_column_by_missing_percentage(null_value_analysis=null_report)

Row(total_row=379989, null_row_company_public_response=209815, missing_percentage=55.21607204419075, column_name='company_public_response')
Row(total_row=379989, null_row_consumer_consent_provided=101388, missing_percentage=26.681825000197374, column_name='consumer_consent_provided')
Row(total_row=379989, null_row_tags=346513, missing_percentage=91.19027129732703, column_name='tags')
